In [1]:
import music21
from music21 import converter, instrument, note, chord
import tensorflow as tf
from tensorflow import keras
import numpy as np
import random
import glob
from fractions import Fraction

In [2]:
default_offset = 0.25

In [3]:
def read_midis(file):
    notes = []
    offsets = []
    midi = converter.parse(file)
    notes_to_parse = midi.flat.notes
    for i in range(len(notes_to_parse)):
        if i != len(notes_to_parse) - 1:
            offsets.append(notes_to_parse[i+1].offset-notes_to_parse[i].offset)
        else:
            offsets.append(default_offset)
        if isinstance(notes_to_parse[i], note.Note):
            notes.append(str(notes_to_parse[i].pitch))
        elif isinstance(notes_to_parse[i], chord.Chord):
            notes.append('.'.join(str(n) for n in notes_to_parse[i].normalOrder))
    return notes,offsets

In [4]:
midi_files = [file for file in glob.glob("midis/*.mid")]
random.shuffle(midi_files)

final_notes,final_offsets = map(list,zip(*[read_midis(file) for file in midi_files]))

all_notes = []
for note_array in final_notes:
    for elem in note_array:
        all_notes.append(elem)
all_notes = np.array(all_notes)

all_offsets = []
for offset_array in final_offsets:
    for indivOff in offset_array:
        all_offsets.append(float(indivOff))
all_offsets = np.around(np.array(all_offsets),3)

In [5]:
note_offset_pairs = list(zip(all_notes, all_offsets))
n_vocab = len(set(note_offset_pairs))

In [6]:
n_vocab

240

In [7]:
pairings = sorted(set(item for item in note_offset_pairs))
sequence_length = 60
pair_to_int = dict((pair, number) for number, pair in enumerate(pairings))

In [8]:
network_in = []
network_out = []
for i in range(0, len(all_notes) - sequence_length):
    seq_in = note_offset_pairs[i:i+sequence_length]
    seq_out = note_offset_pairs[i + sequence_length]
    network_in.append([pair_to_int[elem] for elem in seq_in])
    network_out.append(pair_to_int[seq_out])

n_patterns = len(network_in)
network_in_array = np.reshape(network_in, (n_patterns, sequence_length,1)) / float(n_vocab)
network_out = keras.utils.to_categorical(network_out)

In [9]:
n_patterns

6235

In [10]:
model = keras.Sequential()
model.add(keras.layers.Dropout(0.8))
model.add(keras.layers.LSTM(256, input_shape = (network_in_array.shape[1], network_in_array.shape[2]), return_sequences = True))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.LSTM(128))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(n_vocab, activation = 'softmax'))

In [11]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')
history = model.fit(network_in_array, network_out, epochs = 100)

Epoch 1/100
195/195 [==============================] - 6s 12ms/step - loss: 4.8687
Epoch 2/100
195/195 [==============================] - 2s 12ms/step - loss: 4.5692
Epoch 3/100
195/195 [==============================] - 2s 12ms/step - loss: 4.5813
Epoch 4/100
195/195 [==============================] - 2s 12ms/step - loss: 4.5135
Epoch 5/100
195/195 [==============================] - 2s 12ms/step - loss: 4.3302
Epoch 6/100
195/195 [==============================] - 2s 12ms/step - loss: 4.2564
Epoch 7/100
195/195 [==============================] - 2s 12ms/step - loss: 4.2059
Epoch 8/100
195/195 [==============================] - 2s 13ms/step - loss: 4.1970
Epoch 9/100
195/195 [==============================] - 2s 12ms/step - loss: 4.1801
Epoch 10/100
195/195 [==============================] - 2s 12ms/step - loss: 4.1201
Epoch 11/100
195/195 [==============================] - 2s 12ms/step - loss: 4.0868
Epoch 12/100
195/195 [==============================] - 2s 12ms/step - loss: 4.0713
E

195/195 [==============================] - 2s 12ms/step - loss: 2.4129
Epoch 99/100
195/195 [==============================] - 2s 12ms/step - loss: 2.4123
Epoch 100/100
195/195 [==============================] - 2s 12ms/step - loss: 2.3433


In [12]:
start = np.random.randint(0, len(network_in)-1)
int_to_pair = dict((number,pair) for number, pair in enumerate(pairings))

pattern = network_in[start]
pred_out = []

for note_index in range(500):
    pred_in = np.reshape(pattern, (1, len(pattern),1)) / float(n_vocab)
    prediction = model.predict(pred_in, verbose = 0)
    
    index = np.random.choice(np.arange(n_vocab), p=prediction.reshape(n_vocab))
    result = int_to_pair[index]
    pred_out.append(result)
    
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
    
offset = 0
output_notes = []

for pattern in pred_out:
    if ('.' in pattern[0]) or pattern[0].isdigit():
        notes_in_chord = pattern[0].split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.ElectricGuitar()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    else:
        new_note = note.Note(pattern[0])
        new_note.offset = offset
        new_note.storedInstrument = instrument.ElectricGuitar()
        output_notes.append(new_note)
    offset += pattern[1]

In [13]:
midi_stream = music21.stream.Stream(output_notes)
midi_stream.write('midi', fp = 'test_out.mid')

'test_out.mid'